# Group 31: Applied Machine Learning - Project 1

> Manoj Bhandari

> Harsh Sharma

## Project: Regression Dataset

In this project we take data of Airbnb listings in the US and try to predict the price of stay in that listing. 

The source of the dataset: https://www.kaggle.com/stevezhenghp/airbnb-price-prediction#train.csv 

The data includes 74411 listings and 29 columns - including log_price, what we are trying to predict. 

There was no information about this data so we assume that since all the listings are in the us, the price (or log_price) that we are trying to predict is the general pricing per 1 night stay of the listing, in USD, not for specific dates/seasons and not including additional fees, i.e. cleaning and airbnb service fees.

The data is for 6 cities across the US: NYC, LA, San Francisco, Washington DC, Boston and Chicago. But in this notebook, we use only the listings in Boston city.

In [1]:
import warnings
warnings.filterwarnings('ignore')

__Import required packages__

In [2]:
# Import required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_rows = 100
pd.options.display.max_columns = 150

In [42]:
import pickle

__Load the airbnb dataset__

In [3]:
raw_data = pd.read_csv("airbnb.csv")

__Subset the dataset to contain only the instances of Boston city__

In [4]:
# Subset the data to Boston city
boston_data = raw_data[raw_data['city'] == 'Boston']
boston_data.reset_index(inplace=True, drop=True)

## Data Pre-processing

__Inspect the dataset to understand the data and recognize any data inconsistencies__

In [5]:
boston_data.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,14648556,4.595120,Townhouse,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,2.0,Real Bed,strict,True,Boston,This is a nice duplex in a good location.Recen...,2016-07-16,t,t,100%,2014-07-27,f,2017-02-07,42.339194,-71.049672,"Comfy room (C) near T, convention center, down...",South Boston,12,88.0,https://a0.muscache.com/im/pictures/176088bb-3...,02127,1.0,1.0
1,4680055,4.682131,Condominium,Private room,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,strict,True,Boston,Tourists/Conference-goers great choice! Privat...,2016-03-20,t,t,100%,2013-06-16,t,2017-09-17,42.330628,-71.053148,Private Bedroom Close To Downtown/Subway Red line,South Boston,40,96.0,https://a0.muscache.com/im/pictures/aad0eaa7-a...,02127,1.0,1.0
2,4274462,4.828314,Apartment,Entire home/apt,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",6,1.0,Real Bed,strict,True,Boston,"An Entire 2 bedroom, 600sqft, apartment w/ 4 t...",2017-09-14,t,t,100%,2015-01-25,f,2017-10-02,42.336007,-71.052918,**NEW*Downtown/Convention/Subway/Beach C130,South Boston,5,100.0,https://a0.muscache.com/im/pictures/3d35ea0b-e...,02127,2.0,4.0
3,2278299,4.094345,House,Private room,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",2,1.0,Real Bed,flexible,False,Boston,"This is a beautiful space in a gorgeous, newly...",NaN,t,f,NaN,2013-12-13,f,NaN,42.319265,-71.113246,"Beautiful Home in Jamaica Plain, MA",Jamaica Plain,0,NaN,https://a0.muscache.com/im/pictures/86832250/f...,02130,1.0,1.0
4,16253186,4.962845,Apartment,Entire home/apt,"{TV,""Wireless Internet"",Kitchen,""Family/kid fr...",2,1.0,Real Bed,flexible,False,Boston,Nicely decorated comfortable 1 bedroom in very...,2017-05-21,t,f,NaN,2017-05-05,t,2017-05-27,42.357198,-71.071588,Clean upscale apt and location,Beacon Hill,2,80.0,https://a0.muscache.com/im/pictures/9bf5ae4c-f...,02114,1.0,1.0


__After initial data inspection, removing the attributes which do not contribute towards the regression__

In [6]:
drop_cols = ['id','city', 'description', 'first_review', 'last_review', 'host_since', 'latitude', 'longitude', \
            'name', 'neighbourhood', 'thumbnail_url', 'zipcode']

boston_data.drop(columns=drop_cols, inplace=True)

__Check the datatype of the retained attributes__

In [7]:
boston_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3468 entries, 0 to 3467
Data columns (total 17 columns):
log_price                 3468 non-null float64
property_type             3468 non-null object
room_type                 3468 non-null object
amenities                 3468 non-null object
accommodates              3468 non-null int64
bathrooms                 3462 non-null float64
bed_type                  3468 non-null object
cancellation_policy       3468 non-null object
cleaning_fee              3468 non-null bool
host_has_profile_pic      3468 non-null object
host_identity_verified    3468 non-null object
host_response_rate        2887 non-null object
instant_bookable          3468 non-null object
number_of_reviews         3468 non-null int64
review_scores_rating      2820 non-null float64
bedrooms                  3465 non-null float64
beds                      3466 non-null float64
dtypes: bool(1), float64(5), int64(2), object(9)
memory usage: 437.0+ KB


__Check for attributes with missing values__

In [8]:
boston_data.isnull().sum()

log_price                   0
property_type               0
room_type                   0
amenities                   0
accommodates                0
bathrooms                   6
bed_type                    0
cancellation_policy         0
cleaning_fee                0
host_has_profile_pic        0
host_identity_verified      0
host_response_rate        581
instant_bookable            0
number_of_reviews           0
review_scores_rating      648
bedrooms                    3
beds                        2
dtype: int64

## Initial impressions on data and next steps

### The following columns have missing values:

> bathrooms

> host_response_rate

> review_scores_rating

> bedrooms

> beds

* Based on analysis, missing values will be imputed with appropriate values

* The column __amenities__ has values in the form of JSON. So we will replace it with the count of amenities provided by the host

* Categorical columns need processing to convert them into numerical form.

__host_has_profile_pic__ , __host_identity_verified__ and __instant_bookable__ columns have t for true and f for false. Replacing t with 1 and f with 0

In [9]:
boston_data.replace(to_replace = "t", value = 1,inplace=True) 
boston_data.replace(to_replace = "f", value = 0,inplace=True)

boston_data.replace(to_replace = True, value = 1,inplace=True) 
boston_data.replace(to_replace = False, value = 0,inplace=True)

__Inspect the distinct values of the column `property_type`__

In [10]:
boston_data['property_type'].value_counts()

Apartment             2383
House                  563
Condominium            339
Townhouse               54
Other                   44
Loft                    24
Bed & Breakfast         17
Boat                    10
Guest suite              8
Villa                    6
Hostel                   4
Guesthouse               4
In-law                   4
Dorm                     3
Timeshare                3
Serviced apartment       1
Boutique hotel           1
Name: property_type, dtype: int64

__Since the occurence of many property types is less than 50(or less than 10 in majority cases), we will replace the values of these property types as `Other` to avoid increased dimensionality__

In [11]:
p_types = ['Apartment','House','Condominium','Townhouse']
boston_data.loc[~boston_data.property_type.isin(p_types), 'property_type'] = 'Other'

__Inspect the distinct values of the column `room_type`__

In [12]:
boston_data['room_type'].unique()

array(['Private room', 'Entire home/apt', 'Shared room'], dtype=object)

__Inspect the distinct values of the column `bed_type`__

In [13]:
boston_data['bed_type'].unique()

array(['Real Bed', 'Futon', 'Airbed', 'Pull-out Sofa', 'Couch'],
      dtype=object)

__Inspect the distinct values of the column `cancellation_policy`__

In [14]:
boston_data['cancellation_policy'].unique()

array(['strict', 'flexible', 'super_strict_30', 'moderate',
       'super_strict_60'], dtype=object)

Since all these features are nominal categorical values, we will use OneHotEncoder to convert these categorical features to numeric features

__Converting the categorical columns to numeric by using `OneHotEncoder`__

In [15]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
feature_df = pd.DataFrame(encoder.fit_transform(boston_data[['room_type','bed_type','cancellation_policy', 'property_type','cleaning_fee']]).toarray(), 
                        columns = encoder.get_feature_names(['room_type','bed_type','cancellation_policy', 'property_type','cleaning_fee']))
boston_data = pd.merge(boston_data, feature_df, how='left', left_index=True, right_index=True)
boston_data.drop(columns=['room_type','bed_type','cancellation_policy', 'property_type','cleaning_fee'], inplace=True)

__Since the column `amenities` has json values, we fetch the count of amenities provided by each property and replace the json values with the count of amenities.__

__We are not using `OneHotEncoder` for this column since the maximum number of amenities is 78 and it will add to the dimensionality of the dataset significantly.__

__In our opinion, the count of amenities is also a good indicator to predict the price rather than taking into account 78 distinct columns.__

In [16]:
amenities_count = []
for i in boston_data['amenities']:
    amenities_count.append(len(i.split(',')))
boston_data['amenities'] = amenities_count

__Inspecting the values of the column `host_response_rate`__

In [17]:
boston_data['host_response_rate'].unique()

array(['100%', nan, '88%', '96%', '92%', '94%', '99%', '93%', '54%',
       '33%', '80%', '70%', '67%', '81%', '90%', '98%', '25%', '86%',
       '97%', '0%', '50%', '75%', '87%', '60%', '77%', '46%', '55%',
       '59%', '83%', '79%', '89%', '64%', '10%', '73%', '68%', '95%',
       '20%', '56%', '78%'], dtype=object)

__Since the response rates are in string, we convert it to numeric values between 0 to 1, where range between 0 and 1 represent values between 0% to 100%__

In [18]:
boston_data['host_response_rate'] = boston_data['host_response_rate'].str.strip('%').astype(float)
boston_data['host_response_rate'] = boston_data['host_response_rate']/100

__Inspect the datatypes of all the columns after data pre-processing__

In [19]:
boston_data.dtypes

log_price                              float64
amenities                                int64
accommodates                             int64
bathrooms                              float64
host_has_profile_pic                     int64
host_identity_verified                   int64
host_response_rate                     float64
instant_bookable                         int64
number_of_reviews                        int64
review_scores_rating                   float64
bedrooms                               float64
beds                                   float64
room_type_Entire home/apt              float64
room_type_Private room                 float64
room_type_Shared room                  float64
bed_type_Airbed                        float64
bed_type_Couch                         float64
bed_type_Futon                         float64
bed_type_Pull-out Sofa                 float64
bed_type_Real Bed                      float64
cancellation_policy_flexible           float64
cancellation_

__After data pre-processing, we have all the columns in numeric form. Now we can proceed to treat missing variables__

## Missing value treatment

In [217]:
boston_data.describe()

,log_price,amenities,accommodates,bathrooms,host_has_profile_pic,host_identity_verified,host_response_rate,instant_bookable,number_of_reviews,review_scores_rating,bedrooms,beds,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,property_type_Apartment,property_type_Condominium,property_type_House,property_type_Other,property_type_Townhouse,cleaning_fee_0.0,cleaning_fee_1.0
count,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.00000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000,2469.000000
mean,4.872628,21.154719,3.387606,1.244431,0.998785,0.646416,0.974062,0.383556,34.621709,93.744836,1.348319,1.829081,0.622924,0.36695,0.010126,0.009316,0.000810,0.007695,0.005670,0.976509,0.162819,0.264885,0.559336,0.012151,0.000810,0.663021,0.098015,0.186715,0.034427,0.017821,0.164844,0.835156
std,0.644995,7.961997,2.281458,0.528267,0.034844,0.478179,0.098226,0.486350,49.685195,7.522178,0.913266,1.385540,0.484752,0.48207,0.100135,0.096086,0.028456,0.087403,0.075103,0.151488,0.369275,0.441361,0.496567,0.109581,0.028456,0.472773,0.297396,0.389762,0.182360,0.132327,0.371115,0.371115
min,2.833213,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,20.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.382027,16.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,4.000000,91.000000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,4.905275,20.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,15.000000,96.000000,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,5.293305,25.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,44.000000,99.000000,2.000000,2.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,7.244228,58.000000,16.000000,6.000000,1.000000,1.000000,1.000000,1.000000,380.000000,100.000000,10.000000,16.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [21]:
missing_columns = boston_data.isnull().sum()
print(missing_columns[missing_columns > 0])

bathrooms                 6
host_response_rate      581
review_scores_rating    648
bedrooms                  3
beds                      2
dtype: int64


__We will remove the rows with missing values since imputation does not makes sense for these features__

In [218]:
boston_data = boston_data.dropna()

In [23]:
boston_data.shape

(2469, 32)

__After data cleaning and pre-processing, we have a dataset with 2469 rows and 32 columns.__

## Data Split and Feature Scaling

__Separating the target variable from the features__

In [24]:
x = boston_data.drop(columns='log_price', axis=1)
y = boston_data['log_price']

__Since we have many columns with values 1 and 0, using MinMaxScaler will ensure all the features are in the range of 0 to 1 bringing all the columns to the same scale__

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.3)

scaler = MinMaxScaler()
x_train_mod = scaler.fit_transform(x_train)
x_test_mod = scaler.transform(x_test)

# Regression Methods

__Setting the model scoring(evaluation) metrics to pass as a parameter to GridSearchCV method__

__For all the regression models, we will be using neg_mean_squared_error as a metric to evaluate the prediction accuracy of the model. In this metric, the lower value is better for prediction accuracy__

__We are not using r-squared value, as it indicates the prediciton power. But we are interested in minimizing the prediction error, so we proceed with MSE__

__Although sklearn uses negative sign to indicate the MSE, the actual MSE is simply the positive version of the number we're getting__

In [93]:
scoring = {'MSE': 'neg_mean_squared_error', 'R_squared': 'r2'}

## KNN Regressor

In [94]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

knn = KNeighborsRegressor()
hyperparams = dict(n_neighbors=np.arange(1,50), p=[1,2])

knn_reg = GridSearchCV(knn, param_grid=hyperparams, scoring=scoring, refit='MSE', cv=10, return_train_score=True)

#Fit the model
grid_search = knn_reg.fit(x_train_mod,y_train)

In [119]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation MSE: {:.2f}".format(grid_search.best_score_))

Best parameters: {'n_neighbors': 16, 'p': 1}
Best cross-validation MSE: -0.18


In [120]:
knn_model = grid_search.best_estimator_

In [121]:
knn_pkl = 'knn.pickle'
pickle.dump(knn_model, open(knn_pkl, 'wb'))

## LinearRegression

In [122]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [221]:
lin_reg.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}

In [225]:
print(f'Train MSE: {mean_squared_error(y_train,lin_reg.predict(x_train_mod))}')
print(f'Test MSE: {mean_squared_error(y_test,lin_reg.predict(x_test_mod))}')

Train MSE: 0.9543791824124208
Test MSE: 0.941664709845328


In [246]:
lin_pkl = 'lin_reg.pickle'
pickle.dump(lin_reg, open(lin_pkl, 'wb'))

## Ridge

In [126]:
from sklearn.linear_model import Ridge

ridge = Ridge()

ridge_params = []
r1 = np.arange(0,1,0.1)
r2 = np.arange(1,101,1)
ridge_params = np.concatenate((r1,r2))

params = dict(alpha=ridge_params)

ridgeCV = GridSearchCV(ridge, param_grid=params, scoring=scoring, refit='MSE', cv=10, return_train_score=True)

#Fit the model
grid_ridge = ridgeCV.fit(x_train_mod,y_train)

/Users/manoj/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [127]:
#Print The value of best Hyperparameters
print("Best parameters: {}".format(grid_ridge.best_params_))
print("Best cross-validation MSE: {:.2f}".format(grid_ridge.best_score_))

Best parameters: {'alpha': 1.0}
Best cross-validation MSE: -0.17


In [128]:
ridge_model = grid_ridge.best_estimator_

In [247]:
ridge_pkl = 'ridge.pickle'
pickle.dump(ridge_model, open(ridge_pkl, 'wb'))

## Lasso

In [130]:
from sklearn.linear_model import Lasso

lasso = Lasso()

lasso_params = []
r1 = np.arange(0,1,0.1)
r2 = np.arange(1,101,1)
lasso_params = np.concatenate((r1,r2))

params = dict(alpha=lasso_params)

lassoCV = GridSearchCV(lasso, param_grid=params, scoring=scoring, refit='MSE', cv=10, return_train_score=True)

#Fit the model
grid_lasso = lassoCV.fit(x_train_mod,y_train)

In [131]:
#Print The value of best Hyperparameters
print("Best parameters: {}".format(grid_lasso.best_params_))
print("Best cross-validation MSE: {:.2f}".format(grid_lasso.best_score_))

Best parameters: {'alpha': 0.0}
Best cross-validation MSE: -0.17


In [132]:
lasso_model = grid_lasso.best_estimator_

In [133]:
lasso_pkl = 'lasso.pickle'
pickle.dump(lasso_model, open(lasso_pkl, 'wb'))

## Polynomial regression

In [134]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline


def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

param_grid = {'polynomialfeatures__degree': np.arange(5)}
grid = GridSearchCV(PolynomialRegression(), param_grid, scoring=scoring, refit='MSE', cv=7)
grid.fit(x_train_mod, y_train)

/Users/manoj/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=7, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('polynomialfeatures',
                                        PolynomialFeatures(degree=2,
                                                           include_bias=True,
                                                           interaction_only=False,
                                                           order='C')),
                                       ('linearregression',
                                        LinearRegression(copy_X=True,
                                                         fit_intercept=True,
                                                         n_jobs=None,
                                                         normalize=False))],
                                verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'polynomialfeatures__degree': array([0, 1, 2, 3, 4])},
             pre_di

In [135]:
model = grid.best_estimator_

In [136]:
poly_pkl = 'polynomial.pickle'
pickle.dump(model, open(poly_pkl, 'wb'))

In [137]:
print("Best parameters: {}".format(grid.best_params_))
print("Best cross-validation MSE: {:.2f}".format(grid.best_score_))

Best parameters: {'polynomialfeatures__degree': 1}
Best cross-validation MSE: -0.17


## SVM

### LinearSVC

In [139]:
from sklearn.svm import LinearSVR

In [160]:
svr_params = []
r1 = np.arange(0.1,1,0.1)
r2 = np.arange(1,101,1)
svr_params = np.concatenate((r1,r2))

param_grid = {'C':svr_params}

In [161]:
linsvr = GridSearchCV(LinearSVR(),param_grid, scoring=scoring, refit='MSE')
linsvr.fit(x_train_mod,y_train)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LinearSVR(C=1.0, dual=True, epsilon=0.0,
                                 fit_intercept=True, intercept_scaling=1.0,
                                 loss='epsilon_insensitive', max_iter=1000,
                                 random_state=None, tol=0.0001, verbose=0),
             iid='warn', n_jobs=None,
             param_grid={'C': array([  0.1,   0.2,   0.3,   0.4,   0.5,   0.6,   0.7,   0.8,   0.9,
         1. ,   2. ,   3. ,   4. ,   5. ,   6. ,   7. ,   8. ,   9. ,
        10. ,  11. ,  12. ,  13. ,...
        46. ,  47. ,  48. ,  49. ,  50. ,  51. ,  52. ,  53. ,  54. ,
        55. ,  56. ,  57. ,  58. ,  59. ,  60. ,  61. ,  62. ,  63. ,
        64. ,  65. ,  66. ,  67. ,  68. ,  69. ,  70. ,  71. ,  72. ,
        73. ,  74. ,  75. ,  76. ,  77. ,  78. ,  79. ,  80. ,  81. ,
        82. ,  83. ,  84. ,  85. ,  86. ,  87. ,  88. ,  89. ,  90. ,
        91. ,  92. ,  93. ,  94. ,  95. ,  96. ,  97. ,

In [162]:
linsvr_model = linsvr.best_estimator_

In [163]:
print("Best parameters: {}".format(linsvr.best_params_))
print("Best cross-validation MSE: {:.2f}".format(linsvr.best_score_))

Best parameters: {'C': 0.6}
Best cross-validation MSE: -0.17


In [164]:
linsvr_pkl = 'linsvr_pkl.pickle'
pickle.dump(linsvr_model, open(linsvr_pkl, 'wb'))

### rbf

In [138]:
from sklearn.svm import SVR

In [192]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf']}

svc_rbf = GridSearchCV(SVR(),param_grid,scoring=scoring, refit='MSE', cv=10)
svc_rbf.fit(x_train_mod,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit='MSE', return_train_score=False,
             scoring={'MSE': 'neg_mean_squared_error', 'R_squared': 'r2'},
             verbose=0)

In [193]:
rbf_model = svc_rbf.best_estimator_

In [ ]:
print("Best parameters: {}".format(svc_rbf.best_params_))
print("Best cross-validation MSE: {:.2f}".format(svc_rbf.best_score_))

In [195]:
rbf_pkl = 'svc_rbf.pickle'
pickle.dump(rbf_model, open(rbf_pkl, 'wb'))

### poly

In [196]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['poly']}

svc_poly = GridSearchCV(SVR(),param_grid,scoring=scoring, refit='MSE', cv=10)
svc_poly.fit(x_train_mod,y_train)

/Users/manoj/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['poly']},
             pre_dispatch='2*n_jobs', refit='MSE', return_train_score=False,
             scoring={'MSE': 'neg_mean_squared_error', 'R_squared': 'r2'},
             verbose=0)

In [197]:
svcpoly_model = svc_poly.best_estimator_

In [198]:
print("Best parameters: {}".format(svc_poly.best_params_))
print("Best cross-validation MSE: {:.2f}".format(svc_poly.best_score_))

Best parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}
Best cross-validation score: -0.16


In [199]:
svc_poly = 'svc_poly.pickle'
pickle.dump(svcpoly_model, open(svc_poly, 'wb'))

### linear

In [200]:
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['linear']}

svc_lin = GridSearchCV(SVR(),param_grid,scoring=scoring, refit='MSE', cv=10)
svc_lin.fit(x_train_mod,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear']},
             pre_dispatch='2*n_jobs', refit='MSE', return_train_score=False,
             scoring={'MSE': 'neg_mean_squared_error', 'R_squared': 'r2'},
             verbose=0)

In [201]:
svclin_model = svc_lin.best_estimator_

In [202]:
print("Best parameters: {}".format(svc_lin.best_params_))
print("Best cross-validation MSE: {:.2f}".format(svc_lin.best_score_))

Best parameters: {'C': 100, 'gamma': 1, 'kernel': 'linear'}
Best cross-validation MSE: -0.17


In [203]:
svc_lin = 'svc_lin.pickle'
pickle.dump(svclin_model, open(svc_lin, 'wb'))

## DesicionTree Regressor

In [204]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()

In [205]:
tree_params = dict(max_depth=np.arange(5,50), max_leaf_nodes=np.arange(50,200))

tree_grid = GridSearchCV(dt, param_grid=tree_params, scoring=scoring, refit='MSE', cv=10)
tree_grid.fit(x_train_mod, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_gr...
       141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
       154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 169, 170, 171, 172, 173, 

In [206]:
dtree_model = tree_grid.best_estimator_

In [207]:
print("Best parameters: {}".format(tree_grid.best_params_))
print("Best cross-validation MSE: {:.2f}".format(tree_grid.best_score_))

Best parameters: {'max_depth': 5, 'max_leaf_nodes': 195}
Best cross-validation MSE: -0.18


In [208]:
dtree_pkl = 'dtree_pkl.pickle'
pickle.dump(dtree_model, open(dtree_pkl, 'wb'))

## Final Model

__From all the above models, we can see that the SVM model with `kernel=rbf` and `kernel=poly` has least neg_mean_squared_error i.e., has least error in prediction when compared to other models.__

__Since the execution time of SVM with kernel=poly is more, it is better to use rbf kernel so that the performance of the model is good.__

# SVM (kernel=rbf)

__The best parameters of the model are:__

In [228]:
print("Best parameters: {}".format(svc_rbf.best_params_))

Best parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


__Get the predictions for train dataset__

In [232]:
train_predictions = rbf_model.predict(x_train_mod)

__Get the predictions for test dataset__

In [233]:
test_predictions = rbf_model.predict(x_test_mod)

__Let us compare the predictions with actual values__

In [237]:
print('The mean squared error on train dataset: {}'.format(mean_squared_error(y_train, train_predictions)))
print('The mean squared error on test dataset: {}'.format(mean_squared_error(y_test, test_predictions)))

The mean squared error on train dataset: 0.12600299938936782
The mean squared error on test dataset: 0.17114134438902392


In [241]:
x_test['predicted_log_price'] = test_predictions
x_test['actual_log_price'] = y_test
x_test['prediction_error'] = x_test['actual_log_price'] - x_test['predicted_log_price']

In [243]:
predictions_df = x_test[['actual_log_price','predicted_log_price','prediction_error']]

In [245]:
predictions_df.head(30)

,actual_log_price,predicted_log_price,prediction_error
3269,4.828314,4.399497,0.428817
1579,4.828314,5.145219,-0.316906
2552,5.680173,5.733593,-0.053420
2953,4.828314,5.268606,-0.440292
2266,4.477337,4.171885,0.305452
3192,4.934474,5.085778,-0.151304
3108,4.317488,4.007599,0.309889
1817,3.688879,4.057736,-0.368857
2505,5.521461,5.058801,0.462660
3325,4.882802,5.007668,-0.124866
